# Preprocessor Notebook : Logements Sociaux, fichier RPLS annuel

Ce notebook traite le fichier Excel du RPLS annuel : données sur les logements sociaux.
Le but est de récupérer les datasets suivants, à partir du fichier XSLX téléchargé depuis le site du ministère du Développement Durable :
 - Données par régions
 - Données par départements
 - Données par EPCI
 - Données par communes

 ### Paramètres
 Ce Notebook prend des paramètres en entrée, définis sur la toute première cellule (ci-dessus).
 La cellule a le tag "parameters" ce qui permet de lui passer des valeurs via papermill.
 - filepath : le chemin vers le fichier Excel à traiter
 - model_name : le nom du modèle source

 ### Principe
 Ce notebook extrait 4 feuilles du fichier Excel d'entrée : region, departement, epci, communes. 
 Chaque feuille est chargée dans un dataFrame puis sauvegardée en .xlsx, et chargée en base de données Bronze.
 Peu de retraitement sur ces dataFrames, seul le tableau "departement" a besoin de renommer une colonne.

## Initialisation

Les cellules suivantes servent à importer les modules nécessaires et à préparer les variables communes utilisées dans les traitements.

In [1]:
# Baseline imports
import pandas as pd
import os
import sys
import datetime

# Dirty trick to be able to import common odis modules, if the notebook is not executed from 13_odis
current_dir = os.getcwd()
parent_dir = os.path.dirname(os.getcwd())
while not current_dir.endswith("13_odis"):
    print("changing to parent dir")
    os.chdir(parent_dir)
    current_dir = parent_dir
    parent_dir = os.path.dirname(current_dir)

print(os.getcwd())
sys.path.append(current_dir)

changing to parent dir
/home/jbn/13_odis


In [2]:
# additional imports
from common.config import load_config
from common.data_source_model import DataSourceModel
from common.utils.file_handler import FileHandler
from common.utils.interfaces.data_handler import OperationType

## Paramètres du Notebook
Paramètres pouvant être passés en input par papermill.

Seuls des types built-in semblent marcher (str, int etc), les classes spécifiques ou les objets mutables (datetime...) semblent faire planter papermill.

Doc officielle de papermill : parametrize [https://papermill.readthedocs.io/en/latest/usage-parameterize.html]

In [3]:
# Define parameters for papermill. 
filepath = 'data/imports/logement_social/logement_social.logements_sociaux_1.xlsx'
model_name = "logement.logements_sociaux"


In [4]:
# Initialize common variables
dataframes = {}
artifacts = []

start_time = datetime.datetime.now(tz=datetime.timezone.utc)
config = load_config("datasources.yaml", response_model=DataSourceModel)
model = config.get_model( model_name = model_name )
# Instantiate File Handler for file loads and dumps
handler = FileHandler()

## Traitement des données
A partir de là, on charge le fichier Excel dans Pandas et on traite les feuilles à récupérer, une par une

In [5]:
# Load workbook to pandas
wb = pd.ExcelFile(
    filepath,
    engine = 'openpyxl'
)

In [6]:
# Load excel sheet for Regions
sheet_name = "REGION"
keep_columns_region = [
    'LIBREG',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]


df_region = pd.read_excel(wb, 
                    sheet_name = "REGION",
                    index_col = "REG",
                    header = 5
                    )

# df_region = df_region[keep_columns_region]
dataframes["REGION"] = df_region

region_artifact = handler.artifact_dump( df_region, "REGION", model)
artifacts.append(region_artifact)

df_region.head()

2025-04-16 17:45:35,291 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_REGION.xlsx'


,LIBREG,nb_loues,nb_vacants,nb_vides,nb_asso,nb_occup_finan,nb_occup_temp,nb_ls,parc_non_conv,nb_lgt_tot,...,ener_A_new,ener_B_new,ener_C_new,ener_D_new,ener_E_new,ener_F_new,ener_G_new,ener_NR_new,nb_dpe_realise,perc_dpe_realise
REG,,,,,,,,,,,,,,,,,,,,,
1,Guadeloupe,35657,1508,1350,161,1383,0,40059,0,40059,...,0,0,0,0,0,0,0,0,0,0.0
2,Martinique,33491,1156,269,19,506,0,35441,0,35441,...,0,0,0,0,0,0,0,0,0,0.0
3,Guyane,19585,1213,405,0,559,0,21762,0,21762,...,0,0,0,0,0,0,0,0,0,0.0
4,La Réunion,80140,1082,1583,188,470,0,83463,0,85944,...,0,0,0,0,0,0,0,0,0,0.0
6,Mayotte,2234,248,78,60,321,0,2941,0,2941,...,0,0,0,0,0,0,0,0,0,0.0


In [7]:
# Load excel sheet for Departments
keep_columns_departments = [
    'Unnamed: 1',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_department = pd.read_excel(wb, 
                    sheet_name = "DEPARTEMENT",
                    index_col = "DEP",
                    header = 5
                    )

# df_department = df_department[keep_columns_departments]

# TODO : rename column for Unnamed: 1

dataframes["DEPARTEMENT"] = df_department

department_artifact = handler.artifact_dump( df_department, "DEPARTEMENT", model)
artifacts.append(department_artifact)

df_department.head()

2025-04-16 17:45:35,727 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_DEPARTEMENT.xlsx'


,Unnamed: 1,nb_loues,nb_vacants,nb_vides,nb_asso,nb_occup_finan,nb_occup_temp,nb_ls,parc_non_conv,nb_lgt_tot,...,ener_A_new,ener_B_new,ener_C_new,ener_D_new,ener_E_new,ener_F_new,ener_G_new,ener_NR_new,nb_dpe_realise,perc_dpe_realise
DEP,,,,,,,,,,,,,,,,,,,,,
01,Ain,46145,1200,1063,203,997,0,49608,1663,51271,...,105,544,5514,3751,1060,170,37,41,31971,64.447267
02,Aisne,37628,1383,1371,220,600,15,41217,170,41387,...,29,299,1912,2769,2870,873,287,453,39397,95.584346
03,Allier,17862,963,810,42,176,0,19853,116,19969,...,29,131,2000,2983,1136,473,176,1,16974,85.498413
04,Alpes-de-Haute-Provence,7549,159,90,22,68,0,7888,44,7932,...,116,162,291,375,546,69,16,1,7604,96.399594
05,Hautes-Alpes,7607,298,30,87,28,0,8050,8,8058,...,3,46,226,401,332,81,10,1,4556,56.596273


In [8]:
# Load excel sheet for EPCI
keep_columns_epci = [
    'LIBEPCI',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_epci = pd.read_excel(wb, 
                    sheet_name = "EPCI",
                    index_col = "EPCI_DEP",
                    header = 5
                    )

# df_epci = df_epci[keep_columns_epci]

dataframes["EPCI"] = df_epci

epci_artifact = handler.artifact_dump( df_epci, "EPCI", model)
artifacts.append(epci_artifact)

df_epci.head()

2025-04-16 17:45:40,854 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_EPCI.xlsx'


,DEP,LIBEPCI,nb_loues,nb_vacants,nb_vides,nb_asso,nb_occup_finan,nb_occup_temp,nb_ls,parc_non_conv,...,ener_A_new,ener_B_new,ener_C_new,ener_D_new,ener_E_new,ener_F_new,ener_G_new,ener_NR_new,nb_dpe_realise,perc_dpe_realise
EPCI_DEP,,,,,,,,,,,,,,,,,,,,,
200029999 - (01),01,CC Rives de l'Ain - Pays du Cerdon,600,17,6,0,14,0,637,0,...,0,6,89,75,9,3,0,0,547,85.871272
200040350 - (01),01,CC Bugey Sud,1758,65,76,0,37,0,1936,22,...,0,26,184,152,71,21,0,0,1063,54.907025
200040590 - (01),01,CA Villefranche Beaujolais Saône,688,13,5,0,7,0,713,14,...,0,4,31,67,14,0,0,0,370,51.893408
200042497 - (01),01,CC Dombes Saône Vallée,1899,29,21,0,27,0,1976,37,...,2,8,76,93,33,1,0,0,1334,67.510121
200042935 - (01),01,CA Haut - Bugey Agglomération,7320,285,311,34,228,0,8178,115,...,20,2,300,414,175,39,3,0,3814,46.637320


In [9]:
# Load excel sheet for COMMUNES
keep_columns_communes = [
    'LIBCOM_DEP',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_communes = pd.read_excel(wb, 
                    sheet_name = "COMMUNES",
                    index_col = "DEPCOM_ARM",
                    header = 5
                    )

# df_communes = df_communes[keep_columns_communes]

dataframes["COMMUNES"] = df_communes

communes_artifact = handler.artifact_dump( df_communes, "COMMUNES", model )
artifacts.append(communes_artifact)

df_communes.head()

2025-04-16 17:47:01,870 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_COMMUNES.xlsx'


,REG,DEP,LIBCOM_DEP,LIBCOM,nb_loues,nb_vacants,nb_vides,nb_asso,nb_occup_finan,nb_occup_temp,...,ener_A_new,ener_B_new,ener_C_new,ener_D_new,ener_E_new,ener_F_new,ener_G_new,ener_NR_new,nb_dpe_realise,perc_dpe_realise
DEPCOM_ARM,,,,,,,,,,,,,,,,,,,,,
01001,84,01,L'Abergement-Clémenciat (01),L'Abergement-Clémenciat,31.0,1.0,0.0,0.0,0.0,0.0,...,0,14,0,4,2,4,0,0,32,100.000000
01004,84,01,Ambérieu-en-Bugey (01),Ambérieu-en-Bugey,1906.0,96.0,59.0,1.0,47.0,0.0,...,27,13,60,0,1,1,0,0,847,40.161214
01005,84,01,Ambérieux-en-Dombes (01),Ambérieux-en-Dombes,107.0,3.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,81,71.681416
01007,84,01,Ambronay (01),Ambronay,120.0,3.0,0.0,0.0,6.0,0.0,...,0,0,28,2,0,0,0,0,42,32.558140
01008,84,01,Ambutrix (01),Ambutrix,16.0,1.0,0.0,0.0,0.0,0.0,...,0,4,0,0,0,0,0,0,17,100.000000


## Sauvegarde des métadonnées
On sauvegarde les métadonnées du processus localement, pour garder l'historique et pouvoir reprendre après erreur si besoin

In [10]:
for artifact in artifacts:
    print(artifact.model_dump( mode = "json" ))

preprocess_metadata = handler.dump_metadata(
    model = model,
    operation = OperationType.PREPROCESS,
    start_time = start_time,
    complete = True,
    errors = 0,
    artifacts = artifacts,
    pages = []
)

{'name': 'REGION', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_REGION.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'DEPARTEMENT', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_DEPARTEMENT.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'EPCI', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_EPCI.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'COMMUNES', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_COMMUNES.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
2025-04-16 17:47:01,921 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_soc

## Chargement en couche Bronze
On charge un engine SQLAchemy pour charger tous les datasets en base

In [12]:
from dotenv import dotenv_values
import sqlalchemy

# prepare db client
vals = dotenv_values()

conn_str = "postgresql://{}:{}@{}:{}/{}".format(
    vals["PG_DB_USER"],
    vals["PG_DB_PWD"],
    vals["PG_DB_HOST"],
    vals["PG_DB_PORT"],
    vals["PG_DB_NAME"]
)

dbengine = sqlalchemy.create_engine(conn_str)

ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
# insert all to bronze
# make the final table name lowercase to avoid issues in Postgre
for name, dataframe in dataframes.items():
    dataframe.to_sql(
        name = f"{model.table_name}_{name.lower()}",
        con = dbengine,
        schema = 'bronze',
        index = True,
        if_exists = 'replace'
    )
